In [ ]:
import chromadb
from openapiutils import download_openapi_spec, get_function_names_from_file, create_function_doc, clean_url, generate_hash, create_function_docs

chroma_client = chromadb.PersistentClient(path="agentprotocol/vectordb")


In [ ]:
doc2 = f'''
Function:
def give_joke(category : str):
"""
Joke categories. Supports: Any, Misc, Programming, Dark, Pun, Spooky, Christmas.
"""
'''

In [ ]:
openapi_url = "https://raw.githubusercontent.com/open-meteo/open-meteo/main/openapi.yml"
service_url = "https://api.open-meteo.com"
output_dir = generate_hash(clean_url(openapi_url))
download_openapi_spec(openapi_url, service_url)

In [ ]:
import subprocess

# Construct the command using string formatting
command = f"openapi-python-generator openapi.json {output_dir}/"  # !openapi-python-generator openapi.json ./api_specification_main/
subprocess.run(command.split(), check=True)

In [ ]:
import os
os.remove("openapi.yml")
os.remove("openapi.json")

In [ ]:
function_docs = create_function_docs(output_dir)

In [ ]:
doc_ids = []
doc_metadatas = []
for function_doc in function_docs:
    doc_ids.append(str(generate_hash(function_doc)))
    doc_metadatas.append({"uri": "", "type": "native"})
    print(str(generate_hash(function_doc)))

In [ ]:
collection = chroma_client.get_or_create_collection(name="tools")
collection.add(
    documents=function_docs,
    metadatas=doc_metadatas,
    ids=doc_ids
)


In [ ]:
user_query = "Hey how is the current weather and windspeed in New York?"

In [ ]:
prompt = f'''
    Function:
    no_op():
    """
    Default OOD
    """
    '''

In [ ]:
results = collection.query(
    query_texts=[user_query], 
    n_results=2, # how many results to return
    # include=["embeddings", "metadatas", "documents", "distances"]
    include=["documents"]
)

for items in results["documents"]:
    for item in items:
        # print(str(item))
        prompt += str(item) + "\n"

prompt = prompt + f'''User Query: {user_query}<human_end>'''
print(prompt)


In [ ]:
from utils import query_raven
call = query_raven(prompt)
print (call)

In [ ]:
eval(call)

In [ ]:
count = len(chroma_client.list_collections())
print(count)
if count > 0:
    print(chroma_client.list_collections())

In [ ]:
chroma_client.delete_collection("tools")

In [ ]:
directory_path = 'a41cc9d3b6585b73a594eb1cb0cab5fcd3f67dad09b45c4572583da07b3cc751'
import shutil
if os.path.exists(directory_path):
    shutil.rmtree(directory_path)